# Cleaned AHP Calculation

In [10]:
import pandas as pd
import numpy as np
from scipy.linalg import eig

# Random Index (RI) values based on matrix size (1 to 10 criteria)
RI = {1: 0.0, 2: 0.0, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}

default_ci_threshold = 0.3

# Response scale AHP (saaty scale)
response_scale_ahp = {
    "A mutlak lebih penting dibandingkan B": 9,
    "A sangat lebih penting dibandingkan B": 7,
    "A cukup lebih penting dibandingkan B": 5,
    "A sedikit lebih penting dibandingkan B": 3,
    "Sama pentingnya": 1,
    "B sedikit lebih penting dibandingkan A": 1/3,
    "B cukup lebih penting dibandingkan A": 1/5,
    "B sangat lebih penting dibandingkan A": 1/7,
    "B mutlak lebih penting dibandingkan A": 1/9
}

balanced_scale = lambda x: (9 + x)/(11-x)


# Response scale balanced
response_scale_balanced = {
    "A mutlak lebih penting dibandingkan B": balanced_scale(9),
    "A sangat lebih penting dibandingkan B": balanced_scale(7),
    "A cukup lebih penting dibandingkan B": balanced_scale(5),
    "A sedikit lebih penting dibandingkan B": balanced_scale(3),
    "Sama pentingnya": 1.0,
    "B sedikit lebih penting dibandingkan A": 1/balanced_scale(3),
    "B cukup lebih penting dibandingkan A": 1/balanced_scale(5),
    "B sangat lebih penting dibandingkan A": 1/balanced_scale(7),
    "B mutlak lebih penting dibandingkan A": 1/balanced_scale(9),
}

# Criteria definitions
ppt_criteria = ["People", "Process", "Technology"]
ppt_criteria_pairs = [
    "3. A: People. B: Process",
    "4. A: People. B: Technology",
    "5. A: Process. B: Technology"
]

people_criteria = [
    "Dukungan manajemen senior",
    "Alokasi sumber daya",
    "Kompatibilitas Teknologi",
    "Perubahan manajemen dan Adaptasi User"
]

people_criteria_pairs = [
    f"7. A: {people_criteria[0]}. B: {people_criteria[1]}",
    f"8. A: {people_criteria[0]}. B: {people_criteria[2]}",
    f"9. A: {people_criteria[0]}. B: {people_criteria[3]}",
    f"10. A: {people_criteria[1]}. B: {people_criteria[2]}",
    f"11. A: {people_criteria[1]}. B: {people_criteria[3]}",
    f"12. A: {people_criteria[2]}. B: {people_criteria[3]}"
]

process_criteria = [
    "Rekayasa ulang proses bisnis",
    "Kebutuhan sistem yang jelas",
    "Standar operasional yang jelas",
    "Mitigasi dan manajemen risiko"
]

process_criteria_pairs = [
    f"14. A: {process_criteria[0]}. B: {process_criteria[1]}",
    f"15. A: {process_criteria[0]}. B: {process_criteria[2]}",
    f"16. A: {process_criteria[0]}. B: {process_criteria[3]}",
    f"17. A: {process_criteria[1]}. B: {process_criteria[2]}",
    f"18. A: {process_criteria[1]}. B: {process_criteria[3]}",
    f"19. A: {process_criteria[2]}. B: {process_criteria[3]}"
]

technology_criteria = [
    "Keamanan yang kuat",
    "Skalabilitas infrastruktur IT",
    "Integrasi Sistem",
    "Performa dan kecepatan sistem"
]

technology_criteria_pairs = [
    f"21. A: {technology_criteria[0]}. B: {technology_criteria[1]}",
    f"22. A: {technology_criteria[0]}. B: {technology_criteria[2]}",
    f"23. A: {technology_criteria[0]}. B: {technology_criteria[3]}",
    f"24. A: {technology_criteria[1]}. B: {technology_criteria[2]}",
    f"25. A: {technology_criteria[1]}. B: {technology_criteria[3]}",
    f"26. A: {technology_criteria[2]}. B: {technology_criteria[3]}"
]

# Function to build pairwise comparison matrix
def build_matrix(row, pairs, criteria, response_scale):
    n = len(criteria)
    matrix = np.ones((n, n))  # Start with identity matrix
    for i, crit_a in enumerate(criteria):
        for j, crit_b in enumerate(criteria):
            if i < j:
                pairwise_key = next(
                    (pair for pair in pairs if f"A: {crit_a}. B: {crit_b}" in pair), None
                )
                if pairwise_key and pairwise_key in row and row[pairwise_key] in response_scale:
                    weight = response_scale[row[pairwise_key]]
                    matrix[i, j] = weight
                    matrix[j, i] = 1 / weight
    return matrix

# Calculate Consistency Index (CI)
def calculate_ci(matrix):
    n = matrix.shape[0]
    eigvals, _ = eig(matrix)
    lambda_max = max(eigvals.real)
    ci = (lambda_max - n) / (n - 1)
    return ci

# Clean data based on CI threshold
def clean_data(data, pairs, criteria, response_scale=response_scale_ahp, threshold=default_ci_threshold):
    valid_responses = []
    for _, row in data.iterrows():
        matrix = build_matrix(row, pairs, criteria, response_scale)
        ci = calculate_ci(matrix)
        cr = ci / RI[len(criteria)] if len(criteria) in RI else float('inf')
        if cr <= threshold:
            valid_responses.append(row)
        else:
            print(f"Response {row['Response ID']} from {row['1. Profil Responden']} has a CR of {cr}")
    return pd.DataFrame(valid_responses)

# Calculate priority vector
def calculate_priority_vector(matrix):
    normalized_matrix = matrix / matrix.sum(axis=0)
    priority_vector = normalized_matrix.mean(axis=1)
    return priority_vector

In [11]:
# Main Script
file_path = "~/Downloads/export-core_banking_system_csf_survey-2024-11-18-07-48-33.csv"
data = pd.read_csv(file_path)


## Saaty Scale

In [12]:
# Clean and process PPT level
ppt_cleaned_data = clean_data(data, ppt_criteria_pairs, ppt_criteria)
print("ppt_cleaned_data length", len(ppt_cleaned_data))

Response cm3hfyqid03lcsrafxk1xa9pf from Eggy, Pratama, PT BPR Tata Asia has a CR of 4.268157323611629
Response cm3h419ak0bdh5yzibe8h3xs7 from Ariya, Permana, PT BPR Sukma Kemang Agung has a CR of 1.3592905335529903
Response cm3gx51za0bogrpmtu5pxmob4 from Rizal, ., PD BPR Bank Daerah Kabupaten Kediri has a CR of 1.3592905335529903
Response cm3g0nhfp048kifbtkhr9scjw from Yadi, Mulyadi, PT BPR Mitra Parahyangan has a CR of 1.3592905335529872
Response cm3fzzpmu00d4okdcq4doni46 from Krisna, -, PT BPR Mitra Parahyangan has a CR of 1.3592905335529903
ppt_cleaned_data length 5


In [13]:
people_cleaned_data = clean_data(data, people_criteria_pairs, people_criteria)
print("people_cleaned_data length", len(people_cleaned_data))

Response cm3gx51za0bogrpmtu5pxmob4 from Rizal, ., PD BPR Bank Daerah Kabupaten Kediri has a CR of 0.41559933791283094
Response cm3g0nhfp048kifbtkhr9scjw from Yadi, Mulyadi, PT BPR Mitra Parahyangan has a CR of 0.41559933791283454
people_cleaned_data length 8


In [14]:
process_cleaned_data = clean_data(data, process_criteria_pairs, process_criteria)
print("process_cleaned_data length", len(process_cleaned_data))


Response cm3g0nhfp048kifbtkhr9scjw from Yadi, Mulyadi, PT BPR Mitra Parahyangan has a CR of 0.41559933791283654
process_cleaned_data length 9


In [15]:
technology_cleaned_data = clean_data(data, technology_criteria_pairs, technology_criteria)
print("technology_cleaned_data length", len(technology_cleaned_data))

Response cm3h419ak0bdh5yzibe8h3xs7 from Ariya, Permana, PT BPR Sukma Kemang Agung has a CR of 0.41559933791283526
Response cm3gx51za0bogrpmtu5pxmob4 from Rizal, ., PD BPR Bank Daerah Kabupaten Kediri has a CR of 0.41559933791283526
technology_cleaned_data length 8


In [16]:
ppt_priority_vectors = [
    calculate_priority_vector(build_matrix(row, ppt_criteria_pairs, ppt_criteria,response_scale_ahp))
    for _, row in ppt_cleaned_data.iterrows()
]

# Example: Calculate priority for People sub-criteria
people_priority_vectors = [
    calculate_priority_vector(build_matrix(row, people_criteria_pairs, people_criteria,response_scale_ahp))
    for _, row in people_cleaned_data.iterrows()
]

# Calculate priority vectors for Process and Technology sub-criteria
process_priority_vectors = [
    calculate_priority_vector(build_matrix(row, process_criteria_pairs, process_criteria,response_scale_ahp))
    for _, row in process_cleaned_data.iterrows()
]

technology_priority_vectors = [
    calculate_priority_vector(build_matrix(row, technology_criteria_pairs, technology_criteria,response_scale_ahp))
    for _, row in technology_cleaned_data.iterrows()
]

# Output results for all levels with formatted criteria and scores
print("PPT Priority Vectors:")
print("cleaned sample size", len(ppt_cleaned_data))
ppt_priority_mean = np.mean(ppt_priority_vectors, axis=0)
for criteria, score in zip(ppt_criteria, ppt_priority_mean):
    print(f"{criteria}: {score:.4f}")

print("\nPeople Priority Vectors:")
print("cleaned sample size", len(people_cleaned_data))
people_priority_mean = np.mean(people_priority_vectors, axis=0)
for criteria, score in zip(people_criteria, people_priority_mean):
    print(f"{criteria}: {score:.4f}")

print("\nProcess Priority Vectors:")
print("cleaned sample size", len(process_cleaned_data))
process_priority_mean = np.mean(process_priority_vectors, axis=0)
for criteria, score in zip(process_criteria, process_priority_mean):
    print(f"{criteria}: {score:.4f}")

print("\nTechnology Priority Vectors:")
print("cleaned sample size", len(technology_cleaned_data))
technology_priority_mean = np.mean(technology_priority_vectors, axis=0)
for criteria, score in zip(technology_criteria, technology_priority_mean):
    print(f"{criteria}: {score:.4f}")

PPT Priority Vectors:
cleaned sample size 5
People: 0.7476
Process: 0.1470
Technology: 0.1055

People Priority Vectors:
cleaned sample size 8
Dukungan manajemen senior: 0.2461
Alokasi sumber daya: 0.2539
Kompatibilitas Teknologi: 0.2201
Perubahan manajemen dan Adaptasi User: 0.2799

Process Priority Vectors:
cleaned sample size 9
Rekayasa ulang proses bisnis: 0.2429
Kebutuhan sistem yang jelas: 0.2606
Standar operasional yang jelas: 0.2405
Mitigasi dan manajemen risiko: 0.2560

Technology Priority Vectors:
cleaned sample size 8
Keamanan yang kuat: 0.1646
Skalabilitas infrastruktur IT: 0.2865
Integrasi Sistem: 0.2605
Performa dan kecepatan sistem: 0.2884


## Balanced Scale

In [22]:
balanced_scale_threshold = 0.2

In [23]:
# Clean and process PPT level
ppt_cleaned_data = clean_data(data, ppt_criteria_pairs, ppt_criteria, response_scale_balanced, balanced_scale_threshold)
print("ppt_cleaned_data length", len(ppt_cleaned_data))

Response cm3hfyqid03lcsrafxk1xa9pf from Eggy, Pratama, PT BPR Tata Asia has a CR of 2.2481506121431374
Response cm3h419ak0bdh5yzibe8h3xs7 from Ariya, Permana, PT BPR Sukma Kemang Agung has a CR of 0.5003537284614606
Response cm3gx51za0bogrpmtu5pxmob4 from Rizal, ., PD BPR Bank Daerah Kabupaten Kediri has a CR of 0.5003537284614606
Response cm3g0nhfp048kifbtkhr9scjw from Yadi, Mulyadi, PT BPR Mitra Parahyangan has a CR of 0.5003537284614583
Response cm3fzzpmu00d4okdcq4doni46 from Krisna, -, PT BPR Mitra Parahyangan has a CR of 0.5003537284614606
ppt_cleaned_data length 5


In [24]:
people_cleaned_data = clean_data(data, people_criteria_pairs, people_criteria, response_scale_balanced, balanced_scale_threshold)
print("people_cleaned_data length", len(people_cleaned_data))

people_cleaned_data length 10


In [25]:
process_cleaned_data = clean_data(data, process_criteria_pairs, process_criteria, response_scale_balanced, balanced_scale_threshold)
print("process_cleaned_data length", len(process_cleaned_data))


process_cleaned_data length 10


In [26]:
technology_cleaned_data = clean_data(data, technology_criteria_pairs, technology_criteria, response_scale_balanced, balanced_scale_threshold)
print("technology_cleaned_data length", len(technology_cleaned_data))

technology_cleaned_data length 10


In [27]:
ppt_priority_vectors = [
    calculate_priority_vector(build_matrix(row, ppt_criteria_pairs, ppt_criteria,response_scale_balanced))
    for _, row in ppt_cleaned_data.iterrows()
]

# Example: Calculate priority for People sub-criteria
people_priority_vectors = [
    calculate_priority_vector(build_matrix(row, people_criteria_pairs, people_criteria,response_scale_balanced))
    for _, row in people_cleaned_data.iterrows()
]

# Calculate priority vectors for Process and Technology sub-criteria
process_priority_vectors = [
    calculate_priority_vector(build_matrix(row, process_criteria_pairs, process_criteria,response_scale_balanced))
    for _, row in process_cleaned_data.iterrows()
]

technology_priority_vectors = [
    calculate_priority_vector(build_matrix(row, technology_criteria_pairs, technology_criteria,response_scale_balanced))
    for _, row in technology_cleaned_data.iterrows()
]

# Output results for all levels with formatted criteria and scores
print("PPT Priority Vectors:")
print("cleaned sample size", len(ppt_cleaned_data))
ppt_priority_mean = np.mean(ppt_priority_vectors, axis=0)
for criteria, score in zip(ppt_criteria, ppt_priority_mean):
    print(f"{criteria}: {score:.4f}")

print("\nPeople Priority Vectors:")
print("cleaned sample size", len(people_cleaned_data))
people_priority_mean = np.mean(people_priority_vectors, axis=0)
for criteria, score in zip(people_criteria, people_priority_mean):
    print(f"{criteria}: {score:.4f}")

print("\nProcess Priority Vectors:")
print("cleaned sample size", len(process_cleaned_data))
process_priority_mean = np.mean(process_priority_vectors, axis=0)
for criteria, score in zip(process_criteria, process_priority_mean):
    print(f"{criteria}: {score:.4f}")

print("\nTechnology Priority Vectors:")
print("cleaned sample size", len(technology_cleaned_data))
technology_priority_mean = np.mean(technology_priority_vectors, axis=0)
for criteria, score in zip(technology_criteria, technology_priority_mean):
    print(f"{criteria}: {score:.4f}")

PPT Priority Vectors:
cleaned sample size 5
People: 0.6356
Process: 0.1901
Technology: 0.1743

People Priority Vectors:
cleaned sample size 10
Dukungan manajemen senior: 0.2459
Alokasi sumber daya: 0.2571
Kompatibilitas Teknologi: 0.2287
Perubahan manajemen dan Adaptasi User: 0.2683

Process Priority Vectors:
cleaned sample size 10
Rekayasa ulang proses bisnis: 0.2432
Kebutuhan sistem yang jelas: 0.2673
Standar operasional yang jelas: 0.2404
Mitigasi dan manajemen risiko: 0.2491

Technology Priority Vectors:
cleaned sample size 10
Keamanan yang kuat: 0.2006
Skalabilitas infrastruktur IT: 0.2673
Integrasi Sistem: 0.2537
Performa dan kecepatan sistem: 0.2784
